# Introduction - Business problem

The business problem is basically answering the question of where should I invest my savings opening a Gym.
The general feeling is that people that practice Yoga are "fitness junkies" and therefore practice both Yoga and Gym.
Is there is a Yoga Studio, there is a market for a Gym

# Data


I will collect data from the different Yoga Studios and Gyms in the regions around my house. 
And I will check whether there is any place where we have a Yoga Studio but not a Gym around.

# Methodology

I extracted from FourSquare data from both Gym and Yoga STudios in the greater the Hague region.
I plotted both types of business in a map of the region and I checked checked whether a pattern aroused. This pattern was clear
However a more analytical approach clusterizing the business by region would allow me to understand whether the first conclusion is valid.
The number of cluster was taken from the visual approach above.
The conclusion expressed below clearly indicates the places where there are no Gyms but we do see Yoga Studios

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2019.8.28-0
  - defaults/win-64::ca-certificates-2019.8.28-0done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - fo

### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'JTD5V3QAIOH0B2JMUUTYH3CNUH0QSC5CKHEEVTGUUS3L0C04' # your Foursquare ID
CLIENT_SECRET = 'SYJUCZKB1BSNF0OQY231KKDELARQ5RGLFJBQFQKSPYGLT345' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JTD5V3QAIOH0B2JMUUTYH3CNUH0QSC5CKHEEVTGUUS3L0C04
CLIENT_SECRET:SYJUCZKB1BSNF0OQY231KKDELARQ5RGLFJBQFQKSPYGLT345


### I will display all the Gyms and Yoga studios close to my house

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [300]:
address = 'Van Diemenstraat 140, The Hague, Netherlands'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

52.0812939 4.2910622


### First we will find the Gym places around my neighbourghood

In [301]:
search_query = 'Gym'
radius = 5000
print(search_query + ' .... OK!')

Gym .... OK!


#### Define the corresponding URL

In [302]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JTD5V3QAIOH0B2JMUUTYH3CNUH0QSC5CKHEEVTGUUS3L0C04&client_secret=SYJUCZKB1BSNF0OQY231KKDELARQ5RGLFJBQFQKSPYGLT345&ll=52.0812939,4.2910622&v=20180604&query=Gym&radius=5000&limit=30'

#### Send the GET Request and examine the results

In [303]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e88a952c94979001b940f06'},
 'response': {'venues': [{'id': '4bab23a2f964a52081963ae3',
    'name': "Kneet's Gym",
    'location': {'address': 'Cederstraat 1a',
     'lat': 52.07773282119759,
     'lng': 4.269306270876303,
     'labeledLatLngs': [{'label': 'display',
       'lat': 52.07773282119759,
       'lng': 4.269306270876303}],
     'distance': 1540,
     'postalCode': '2565 JM',
     'cc': 'NL',
     'city': 'Den Haag',
     'state': 'Zuid-Holland',
     'country': 'Nederland',
     'formattedAddress': ['Cederstraat 1a', '2565 JM Den Haag', 'Nederland']},
    'categories': [{'id': '4bf58dd8d48988d175941735',
      'name': 'Gym / Fitness Center',
      'pluralName': 'Gyms or Fitness Centers',
      'shortName': 'Gym / Fitness',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586014569',
    'hasPerk': False},
   {'id': '5802fb9cd67c977679e

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [304]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4bab23a2f964a52081963ae3,Kneet's Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'Gym / Fitness Center', 'pluralName': 'Gyms or Fitness Centers', 'shortName': 'Gym / Fitness', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_', 'suffix': '.png'}, 'primary': True}]",v-1586014569,False,Cederstraat 1a,52.077733,4.269306,"[{'label': 'display', 'lat': 52.07773282119759, 'lng': 4.269306270876303}]",1540,2565 JM,NL,Den Haag,Zuid-Holland,Nederland,"[Cederstraat 1a, 2565 JM Den Haag, Nederland]",NaN
1,5802fb9cd67c977679eb3bc9,Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'Gym / Fitness Center', 'pluralName': 'Gyms or Fitness Centers', 'shortName': 'Gym / Fitness', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_', 'suffix': '.png'}, 'primary': True}]",v-1586014569,False,NaN,52.090197,4.282450,"[{'label': 'display', 'lat': 52.090197, 'lng': 4.28245}]",1152,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[The Hague Marriott Hotel, Den Haag, Nederland]",The Hague Marriott Hotel
2,4c9cb0427c096dcbebacc3d1,Stadhuis Den Haag Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'Gym', 'pluralName': 'Gyms', 'shortName': 'Gym', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_', 'suffix': '.png'}, 'primary': True}]",v-1586014569,False,Spui,52.078614,4.317579,"[{'label': 'display', 'lat': 52.07861365610862, 'lng': 4.317579407037812}]",1838,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Spui, Den Haag, Nederland]",NaN
3,4c204eddfec3b7130cd5bec6,The Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'Gym', 'pluralName': 'Gyms', 'shortName': 'Gym', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_', 'suffix': '.png'}, 'primary': True}]",v-1586014569,False,Alexanderplein,52.087989,4.300159,"[{'label': 'display', 'lat': 52.087988893040205, 'lng': 4.300159001119215}]",970,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Alexanderplein (Javastraat), Den Haag, Nederland]",Javastraat
4,5b6c0fc1d69ed0002ca42a76,ODA Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'Gym', 'pluralName': 'Gyms', 'shortName': 'Gym', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_', 'suffix': '.png'}, 'primary': True}]",v-1586014569,False,Gaslaan 200,52.074666,4.284584,"[{'label': 'display', 'lat': 52.074666, 'lng': 4.284584}]",860,2562 SW,NL,Den Haag,Zuid-Holland,Nederland,"[Gaslaan 200, 2562 SW Den Haag, Nederland]",NaN


#### Define information of interest and filter dataframe

In [305]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered


is_gym = dataframe_filtered['categories']=='Gym'

dataframe_filtered = dataframe_filtered[is_gym]
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
2,Stadhuis Den Haag Gym,Gym,Spui,52.078614,4.317579,"[{'label': 'display', 'lat': 52.07861365610862, 'lng': 4.317579407037812}]",1838,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Spui, Den Haag, Nederland]",NaN,4c9cb0427c096dcbebacc3d1
3,The Gym,Gym,Alexanderplein,52.087989,4.300159,"[{'label': 'display', 'lat': 52.087988893040205, 'lng': 4.300159001119215}]",970,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Alexanderplein (Javastraat), Den Haag, Nederland]",Javastraat,4c204eddfec3b7130cd5bec6
4,ODA Gym,Gym,Gaslaan 200,52.074666,4.284584,"[{'label': 'display', 'lat': 52.074666, 'lng': 4.284584}]",860,2562 SW,NL,Den Haag,Zuid-Holland,Nederland,"[Gaslaan 200, 2562 SW Den Haag, Nederland]",NaN,5b6c0fc1d69ed0002ca42a76
7,Hotel Des Indes gym,Gym,NaN,52.083993,4.312857,"[{'label': 'display', 'lat': 52.08399271650959, 'lng': 4.312857433077685}]",1520,NaN,NL,NaN,NaN,Nederland,[Nederland],NaN,5191203a498e2b077fc416c7
9,EA Gym,Gym,NaN,52.091651,4.274929,"[{'label': 'display', 'lat': 52.091651, 'lng': 4.27492877}]",1595,NaN,NL,NaN,NaN,Nederland,[Nederland],NaN,4d18a897bb488cfa2977b6d4
10,Ministerie Van Defensie Gym,Gym,NaN,52.078293,4.315913,"[{'label': 'display', 'lat': 52.078293, 'lng': 4.315913}]",1732,NaN,NL,NaN,NaN,Nederland,[Nederland],NaN,56e6af35498efb2e21339ad7
15,De Gymnastiek Fabriek,Gym,NaN,52.077942,4.301949,"[{'label': 'display', 'lat': 52.07794163398366, 'lng': 4.301948836123276}]",833,NaN,NL,NaN,NaN,Nederland,[Nederland],NaN,523157d411d252500a8d3a50


#### Let's visualize the natural places that are nearby

In [306]:
dataframe_filtered.name

2     Stadhuis Den Haag Gym      
3     The Gym                    
4     ODA Gym                    
7     Hotel Des Indes gym        
9     EA Gym                     
10    Ministerie Van Defensie Gym
15    De Gymnastiek Fabriek      
Name: name, dtype: object

#### Create a map of Gyms around my house

In [382]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent my house
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Home',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Gym Places as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    

    

# display map
venues_map

## We search Yoga places now

In [308]:
search_query = 'Yoga'
radius = 5000
print(search_query + ' .... OK!')

Yoga .... OK!


## Define URL 

In [309]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JTD5V3QAIOH0B2JMUUTYH3CNUH0QSC5CKHEEVTGUUS3L0C04&client_secret=SYJUCZKB1BSNF0OQY231KKDELARQ5RGLFJBQFQKSPYGLT345&ll=52.0812939,4.2910622&v=20180604&query=Yoga&radius=5000&limit=30'

In [310]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e88a88f211536001b69bfa5'},
 'response': {'venues': [{'id': '4bbb8d6751b89c74025c862a',
    'name': 'Bikram Yoga Den Haag',
    'location': {'address': 'Prins Hendrikplein 2',
     'lat': 52.08239299681685,
     'lng': 4.295000597837386,
     'labeledLatLngs': [{'label': 'display',
       'lat': 52.08239299681685,
       'lng': 4.295000597837386}],
     'distance': 295,
     'postalCode': '2518JA',
     'cc': 'NL',
     'city': 'Den Haag',
     'state': 'Zuid-Holland',
     'country': 'Nederland',
     'formattedAddress': ['Prins Hendrikplein 2',
      '2518JA Den Haag',
      'Nederland']},
    'categories': [{'id': '4bf58dd8d48988d102941735',
      'name': 'Yoga Studio',
      'pluralName': 'Yoga Studios',
      'shortName': 'Yoga Studio',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '82721391'},
    'referralId': 'v-1586014593',


In [311]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe2 = pd.json_normalize(venues)
dataframe2.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4bbb8d6751b89c74025c862a,Bikram Yoga Den Haag,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Yoga Studio', 'pluralName': 'Yoga Studios', 'shortName': 'Yoga Studio', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_', 'suffix': '.png'}, 'primary': True}]",v-1586014593,False,Prins Hendrikplein 2,52.082393,4.295001,"[{'label': 'display', 'lat': 52.08239299681685, 'lng': 4.295000597837386}]",295,2518JA,NL,Den Haag,Zuid-Holland,Nederland,"[Prins Hendrikplein 2, 2518JA Den Haag, Nederland]",82721391,NaN
1,4cb494b7ef1b370419fb5900,Linggan Yoga & Lifestyle,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Yoga Studio', 'pluralName': 'Yoga Studios', 'shortName': 'Yoga Studio', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_', 'suffix': '.png'}, 'primary': True}]",v-1586014593,False,Westeinde 12 D,52.076542,4.305828,"[{'label': 'display', 'lat': 52.07654151652661, 'lng': 4.305827972720164}]",1140,2512 HD,NL,Den Haag,Zuid-Holland,Nederland,"[Westeinde 12 D (In Het Spaanse Hof), 2512 HD Den Haag, Nederland]",NaN,In Het Spaanse Hof
2,51a77f0c498e78106d05a264,Diana van Gelder - Yoga & Meditatie,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Yoga Studio', 'pluralName': 'Yoga Studios', 'shortName': 'Yoga Studio', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_', 'suffix': '.png'}, 'primary': True}]",v-1586014593,False,Buitenom,52.060995,4.288571,"[{'label': 'display', 'lat': 52.06099466672304, 'lng': 4.288571106637605}]",2266,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Buitenom, Den Haag, Nederland]",NaN,NaN
3,4b96a809f964a520eddb34e3,Sunshine Yoga,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Yoga Studio', 'pluralName': 'Yoga Studios', 'shortName': 'Yoga Studio', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_', 'suffix': '.png'}, 'primary': True}]",v-1586014593,False,Witte de Withstraat 11,52.080577,4.295279,"[{'label': 'display', 'lat': 52.08057671124644, 'lng': 4.295279274986971}]",299,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Witte de Withstraat 11, Den Haag, Nederland]",NaN,NaN
4,5679319d498e383e1855067e,Delight Yoga,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Yoga Studio', 'pluralName': 'Yoga Studios', 'shortName': 'Yoga Studio', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_', 'suffix': '.png'}, 'primary': True}]",v-1586014593,False,Scheveningseweg 14,52.087202,4.298776,"[{'label': 'display', 'lat': 52.0872015716532, 'lng': 4.298776442034205}]",843,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Scheveningseweg 14, Den Haag, Nederland]",NaN,NaN


In [312]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe2_filtered = dataframe2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe2_filtered['categories'] = dataframe2_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe2_filtered.columns = [column.split('.')[-1] for column in dataframe2_filtered.columns]

dataframe2_filtered
is_yoga = dataframe2_filtered['categories']=='Yoga Studio'

dataframe2_filtered = dataframe2_filtered[is_yoga]
dataframe2_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Bikram Yoga Den Haag,Yoga Studio,Prins Hendrikplein 2,52.082393,4.295001,"[{'label': 'display', 'lat': 52.08239299681685, 'lng': 4.295000597837386}]",295,2518JA,NL,Den Haag,Zuid-Holland,Nederland,"[Prins Hendrikplein 2, 2518JA Den Haag, Nederland]",NaN,4bbb8d6751b89c74025c862a
1,Linggan Yoga & Lifestyle,Yoga Studio,Westeinde 12 D,52.076542,4.305828,"[{'label': 'display', 'lat': 52.07654151652661, 'lng': 4.305827972720164}]",1140,2512 HD,NL,Den Haag,Zuid-Holland,Nederland,"[Westeinde 12 D (In Het Spaanse Hof), 2512 HD Den Haag, Nederland]",In Het Spaanse Hof,4cb494b7ef1b370419fb5900
2,Diana van Gelder - Yoga & Meditatie,Yoga Studio,Buitenom,52.060995,4.288571,"[{'label': 'display', 'lat': 52.06099466672304, 'lng': 4.288571106637605}]",2266,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Buitenom, Den Haag, Nederland]",NaN,51a77f0c498e78106d05a264
3,Sunshine Yoga,Yoga Studio,Witte de Withstraat 11,52.080577,4.295279,"[{'label': 'display', 'lat': 52.08057671124644, 'lng': 4.295279274986971}]",299,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Witte de Withstraat 11, Den Haag, Nederland]",NaN,4b96a809f964a520eddb34e3
4,Delight Yoga,Yoga Studio,Scheveningseweg 14,52.087202,4.298776,"[{'label': 'display', 'lat': 52.0872015716532, 'lng': 4.298776442034205}]",843,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Scheveningseweg 14, Den Haag, Nederland]",NaN,5679319d498e383e1855067e
6,Saktiisha Yoga,Yoga Studio,Javastraat 12D,52.087658,4.302167,"[{'label': 'display', 'lat': 52.087658, 'lng': 4.3021665}]",1038,2585 AN,NL,Den Haag,Zuid-Holland,Nederland,"[Javastraat 12D, 2585 AN Den Haag, Nederland]",NaN,5cdd861173fe25002c66f55b
7,Yoga Studio,Yoga Studio,Anna Palownastraat,52.086910,4.308326,"[{'label': 'display', 'lat': 52.08691025573837, 'lng': 4.308325665398722}]",1336,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Anna Palownastraat, Den Haag, Nederland]",NaN,4e4a3fdae4cd4fd730018aba
10,Samudra Yoga & More,Yoga Studio,Korte Poten 59,52.080084,4.318160,"[{'label': 'display', 'lat': 52.080084, 'lng': 4.31816}]",1858,2511 EC,NL,Den Haag,Zuid-Holland,Nederland,"[Korte Poten 59, 2511 EC Den Haag, Nederland]",NaN,5d13afeea8da670023e7f5e8
12,Yoga,Yoga Studio,NaN,52.052700,4.236296,"[{'label': 'display', 'lat': 52.0527, 'lng': 4.236296}]",4917,NaN,NL,NaN,NaN,Nederland,[Nederland],NaN,5b93d84f25ecca002d8ce7f5
13,Fiore Yoga Scheveningen,Yoga Studio,2e Messstraat,52.092513,4.266997,"[{'label': 'display', 'lat': 52.092513, 'lng': 4.266997}]",2066,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[2e Messstraat, Den Haag, Nederland]",NaN,4e80eccbbe7b2056ab144467


In [383]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent my house
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Home',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


    
    # add the Gym places as blue circle markers
for lat, lng, label in zip(dataframe2_filtered.lat, dataframe2_filtered.lng, dataframe2_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)
    

    # add the Yoga Studios as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    



# display map
venues_map

In the map we see the gyms again in Blue and the yoga studios in Green
We can see clearly in the areas far from the city centre there are Yoga studios and Gyms but the areas far away do not have Gyms 
like Loosduinen, Sheveningen or Haagse Hout

## Analytical study

We will now use an analytic approach to see if we arrive to the same result using CLUSTER and geometry 

### we merge both Gym and Yoga data

In [363]:
frames = [dataframe2_filtered, dataframe_filtered]

result = pd.concat(frames)
result.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Bikram Yoga Den Haag,Yoga Studio,Prins Hendrikplein 2,52.082393,4.295001,"[{'label': 'display', 'lat': 52.08239299681685, 'lng': 4.295000597837386}]",295,2518JA,NL,Den Haag,Zuid-Holland,Nederland,"[Prins Hendrikplein 2, 2518JA Den Haag, Nederland]",NaN,4bbb8d6751b89c74025c862a
1,Linggan Yoga & Lifestyle,Yoga Studio,Westeinde 12 D,52.076542,4.305828,"[{'label': 'display', 'lat': 52.07654151652661, 'lng': 4.305827972720164}]",1140,2512 HD,NL,Den Haag,Zuid-Holland,Nederland,"[Westeinde 12 D (In Het Spaanse Hof), 2512 HD Den Haag, Nederland]",In Het Spaanse Hof,4cb494b7ef1b370419fb5900
2,Diana van Gelder - Yoga & Meditatie,Yoga Studio,Buitenom,52.060995,4.288571,"[{'label': 'display', 'lat': 52.06099466672304, 'lng': 4.288571106637605}]",2266,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Buitenom, Den Haag, Nederland]",NaN,51a77f0c498e78106d05a264
3,Sunshine Yoga,Yoga Studio,Witte de Withstraat 11,52.080577,4.295279,"[{'label': 'display', 'lat': 52.08057671124644, 'lng': 4.295279274986971}]",299,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Witte de Withstraat 11, Den Haag, Nederland]",NaN,4b96a809f964a520eddb34e3
4,Delight Yoga,Yoga Studio,Scheveningseweg 14,52.087202,4.298776,"[{'label': 'display', 'lat': 52.0872015716532, 'lng': 4.298776442034205}]",843,NaN,NL,Den Haag,Zuid-Holland,Nederland,"[Scheveningseweg 14, Den Haag, Nederland]",NaN,5679319d498e383e1855067e


In [364]:
X=result.loc[:,['name','lat','lng', 'categories', 'address']]
print(X)


                                   name        lat       lng   categories  \
0   Bikram Yoga Den Haag                 52.082393  4.295001  Yoga Studio   
1   Linggan Yoga & Lifestyle             52.076542  4.305828  Yoga Studio   
2   Diana van Gelder - Yoga & Meditatie  52.060995  4.288571  Yoga Studio   
3   Sunshine Yoga                        52.080577  4.295279  Yoga Studio   
4   Delight Yoga                         52.087202  4.298776  Yoga Studio   
6   Saktiisha Yoga                       52.087658  4.302167  Yoga Studio   
7   Yoga Studio                          52.086910  4.308326  Yoga Studio   
10  Samudra Yoga & More                  52.080084  4.318160  Yoga Studio   
12  Yoga                                 52.052700  4.236296  Yoga Studio   
13  Fiore Yoga Scheveningen              52.092513  4.266997  Yoga Studio   
14  Yoga Natura                          52.108066  4.282326  Yoga Studio   
15  Gonita Yoga                          52.105182  4.280965  Yoga Studio   

In [365]:
K_clusters = range(1,10)
from sklearn.cluster import KMeans
kmeans = [KMeans(n_clusters=i) for i in K_clusters]

In [384]:
## In the graphic we can see 5 clusters geographicall so we will see that

In [366]:
kmeans = KMeans(n_clusters = 5, init ='k-means++')
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['cluster_label'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point
X.head(29)



,name,lat,lng,categories,address,cluster_label
0,Bikram Yoga Den Haag,52.082393,4.295001,Yoga Studio,Prins Hendrikplein 2,1
1,Linggan Yoga & Lifestyle,52.076542,4.305828,Yoga Studio,Westeinde 12 D,1
2,Diana van Gelder - Yoga & Meditatie,52.060995,4.288571,Yoga Studio,Buitenom,1
3,Sunshine Yoga,52.080577,4.295279,Yoga Studio,Witte de Withstraat 11,1
4,Delight Yoga,52.087202,4.298776,Yoga Studio,Scheveningseweg 14,1
6,Saktiisha Yoga,52.087658,4.302167,Yoga Studio,Javastraat 12D,1
7,Yoga Studio,52.086910,4.308326,Yoga Studio,Anna Palownastraat,1
10,Samudra Yoga & More,52.080084,4.318160,Yoga Studio,Korte Poten 59,1
12,Yoga,52.052700,4.236296,Yoga Studio,NaN,4
13,Fiore Yoga Scheveningen,52.092513,4.266997,Yoga Studio,2e Messstraat,0


We will generate a data frame for each cluster to represent it graphically and see if clusters and neighbourhoods overlap

In [385]:


A = X[X['cluster_label']== 0]
B = X[X['cluster_label']== 1]
C = X[X['cluster_label']== 2]
D = X[X['cluster_label']== 3]
E = X[X['cluster_label']== 4]





In [386]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent my house
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Home',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


    
    # Blue for cluster zero 
for lat, lng, label in zip(A.lat, A.lng, A.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
     
        # yellow for cluster 1
for lat, lng, label in zip(B.lat, B.lng, B.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)

      # brown cluster 2
for lat, lng, label in zip(C.lat, C.lng, C.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='brown',
        popup=label,
        fill = True,
        fill_color='brown',
        fill_opacity=0.6
    ).add_to(venues_map)

    
      # green cluster 3
for lat, lng, label in zip(D.lat, D.lng, D.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

    
       # purple cluster 4
for lat, lng, label in zip(E.lat, E.lng, E.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(venues_map)



# display map
venues_map

The map is clearly clustering the areas by neighbourhoods
- The cluster 0 is Sergebroek and Scheveningen
- the cluster 1 is City Centre
- the cluster 2 is leischendam and haagse hout
- the cluster 3 is Rijskwijk
- the cluster 4 is Lossdouinen

## we group number of gym and yoga so we can count how many are there by cluster

In [369]:
#X['cluster_label','categories'].value_counts()
#X.groupby('cluster_label')['categories'].count()
#X.groupby('cluster_label')['categories'].sum()
X.groupby(['cluster_label', 'categories'])['name'].count()


cluster_label  categories 
0              Gym            1
               Yoga Studio    5
1              Gym            6
               Yoga Studio    9
2              Yoga Studio    2
3              Yoga Studio    2
4              Yoga Studio    1
Name: name, dtype: int64

## Results 


WE can clearly see in the second map that close to my house (marked in red) in the City Centre Neighbourhood there is a strong correlation betewen Yoga studios 
In the table above we see ther are gyms and Yoga Stuidos (6 and 9 respectively)  
In the area of Sergebroek and Scheveningen there are also Gyms and Yoga Studiso (6 and 9 respectively)
However, the areas farher awyay like Rijskwik (cluster 3 -green) and Losduinen (cluser 4 - purple) have only Gyms (2 and 1 as per table above)
By using echonomic studies like price of houses and tax info we know that the area of Rijkswijk has a high economic power has a  Yoga studio but not Gym.




## Discussion 

Seeing the results above expressed, my recomendation is to open a Gym in Rijkswik 



## Conclusion section where you conclude the report.


I would invest in a Yoga studio in the area of Rijkswijk

<a id="item5"></a>